In [2]:
import spacy
from spacy.lang.ko.examples import sentences
from transformers import AutoTokenizer, AutoModelForTokenClassification
dependency_labels = ['NP', 'VP', 'AP', 'DP', 'IP', 'ROOT','NP_SBJ', 'NP_OBJ', 'VP_MOD', 'AP_MOD','NP_AJT', 'VP_AJT', 'AP_AJT', 'DP_AJT']
    
nlp = spacy.load("ko_core_news_lg")
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
klue_model = AutoModelForTokenClassification.from_pretrained(
            "klue/bert-base",
            num_labels=len(dependency_labels)  # 의존관계 레이블 수에 맞게 조정
        )



/home/jaeseok/anaconda3/envs/coshow/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jaeseok/anaconda3/envs/coshow/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# 예시 문장 정의
text1 = "아메리카노 하나는 뜨겁게 하나는 차갑게 주세요"
text2 = "아이스 아메리카노 샷 추가 두잔 주세요"

# SpaCy를 사용한 문장 구조 분석
doc1 = nlp(text1)
doc2 = nlp(text2)

print("=== SpaCy 문장 분석 결과 ===")
print("\n[첫번째 문장 형태소 분석]")
for token in doc1:
    print(f"텍스트: {token.text:<10} 품사: {token.pos_:<10} 세부품사: {token.tag_:<10} 의존관계: {token.dep_:<10} 원형: {token.lemma_:<10}")

print("\n[첫번째 문장 개체명 인식]")
for ent in doc1.ents:
    print(f"텍스트: {ent.text:<10} 레이블: {ent.label_:<10}")

print("\n[두번째 문장 형태소 분석]")
for token in doc2:
    print(f"텍스트: {token.text:<10} 품사: {token.pos_:<10} 세부품사: {token.tag_:<10} 의존관계: {token.dep_:<10} 원형: {token.lemma_:<10}")

print("\n[두번째 문장 개체명 인식]")
for ent in doc2.ents:
    print(f"텍스트: {ent.text:<10} 레이블: {ent.label_:<10}")

# Stanza를 사용한 의존 구문 분석
import stanza
#stanza.download('ko')  # 한국어 모델 다운로드
nlp_stanza = stanza.Pipeline('ko')

# 첫번째 문장 분석
doc_stanza1 = nlp_stanza(text1)
print("\n=== Stanza 의존 구문 분석 결과 (첫번째 문장) ===")
for sent in doc_stanza1.sentences:
    for word in sent.words:
        print(f"단어: {word.text:<10} 품사: {word.upos:<10} 의존관계: {word.deprel:<10} 의존부모: {sent.words[word.head-1].text if word.head > 0 else 'ROOT'}")

# 두번째 문장 분석
doc_stanza2 = nlp_stanza(text2)
print("\n=== Stanza 의존 구문 분석 결과 (두번째 문장) ===")
for sent in doc_stanza2.sentences:
    for word in sent.words:
        print(f"단어: {word.text:<10} 품사: {word.upos:<10} 의존관계: {word.deprel:<10} 의존부모: {sent.words[word.head-1].text if word.head > 0 else 'ROOT'}")

# KLUE/RoBERTa 기반 의존성 파싱
# KLUE/BERT 모델을 사용한 토큰 분류
print("\n=== KLUE/BERT 의존성 분석 결과 (첫번째 문장) ===")
inputs1 = tokenizer(text1, return_tensors="pt", truncation=True, max_length=128)
outputs1 = klue_model(**inputs1)
predictions1 = outputs1.logits.argmax(-1)[0].tolist()
tokens1 = tokenizer.convert_ids_to_tokens(inputs1["input_ids"][0])

for token, pred in zip(tokens1[1:-1], predictions1[1:-1]):
    dependency_label = dependency_labels[pred]
    print(f"토큰: {token:<10} 의존관계: {dependency_label}")

print("\n=== KLUE/BERT 의존성 분석 결과 (두번째 문장) ===")
inputs2 = tokenizer(text2, return_tensors="pt", truncation=True, max_length=128)
outputs2 = klue_model(**inputs2)
predictions2 = outputs2.logits.argmax(-1)[0].tolist()
tokens2 = tokenizer.convert_ids_to_tokens(inputs2["input_ids"][0])

for token, pred in zip(tokens2[1:-1], predictions2[1:-1]):
    dependency_label = dependency_labels[pred]
    print(f"토큰: {token:<10} 의존관계: {dependency_label}")


2024-11-17 01:04:02 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


=== SpaCy 문장 분석 결과 ===

[첫번째 문장 형태소 분석]
텍스트: 아메리카노      품사: PROPN      세부품사: nq         의존관계: compound   원형: 아메리카노     
텍스트: 하나는        품사: NUM        세부품사: nnc+jxt    의존관계: dislocated 원형: 하나+는      
텍스트: 뜨겁게        품사: SCONJ      세부품사: paa+ecs    의존관계: xcomp      원형: 뜨겁+게      
텍스트: 하나는        품사: NUM        세부품사: nnc+jxt    의존관계: dislocated 원형: 하나+는      
텍스트: 차갑게        품사: ADJ        세부품사: paa+ecx    의존관계: ROOT       원형: 차갑+게      
텍스트: 주세요        품사: SCONJ      세부품사: px+ep+ef   의존관계: aux        원형: 주세요       

[첫번째 문장 개체명 인식]
텍스트: 하나는        레이블: QT        
텍스트: 하나는        레이블: QT        

[두번째 문장 형태소 분석]
텍스트: 아이스        품사: PROPN      세부품사: nq         의존관계: compound   원형: 아이스       
텍스트: 아메리카노      품사: PROPN      세부품사: nq         의존관계: compound   원형: 아메리카노     
텍스트: 샷          품사: NOUN       세부품사: ncn        의존관계: compound   원형: 샷         
텍스트: 추가         품사: NOUN       세부품사: ncpa       의존관계: nsubj      원형: 추+가       
텍스트: 두잔         품사: NUM        세부품사: npp        의존관계: dep     

2024-11-17 01:04:02 INFO: Downloaded file to /home/jaeseok/stanza_resources/resources.json
2024-11-17 01:04:03 INFO: Loading these models for language: ko (Korean):
| Processor | Package        |
------------------------------
| tokenize  | kaist          |
| pos       | kaist_nocharlm |
| lemma     | kaist_nocharlm |
| depparse  | kaist_nocharlm |

2024-11-17 01:04:03 INFO: Using device: cpu
2024-11-17 01:04:03 INFO: Loading: tokenize
/home/jaeseok/anaconda3/envs/coshow/lib/python3.8/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functio


=== Stanza 의존 구문 분석 결과 (첫번째 문장) ===
단어: 아메리카노      품사: NOUN       의존관계: compound   의존부모: 하나는
단어: 하나는        품사: NUM        의존관계: dislocated 의존부모: 주세요
단어: 뜨겁게        품사: SCONJ      의존관계: xcomp      의존부모: 하나는
단어: 하나는        품사: NUM        의존관계: dislocated 의존부모: 주세요
단어: 차갑게        품사: ADJ        의존관계: xcomp      의존부모: 주세요
단어: 주세요        품사: VERB       의존관계: root       의존부모: ROOT

=== Stanza 의존 구문 분석 결과 (두번째 문장) ===
단어: 아이스        품사: NOUN       의존관계: compound   의존부모: 아메리카노
단어: 아메리카노      품사: NOUN       의존관계: compound   의존부모: 샷
단어: 샷          품사: NOUN       의존관계: compound   의존부모: 추가
단어: 추가         품사: NOUN       의존관계: nsubj      의존부모: 두잔
단어: 두잔         품사: VERB       의존관계: ccomp      의존부모: 주세요
단어: 주세요        품사: VERB       의존관계: root       의존부모: ROOT

=== KLUE/BERT 의존성 분석 결과 (첫번째 문장) ===
토큰: 아메리카노      의존관계: IP
토큰: 하나         의존관계: NP_SBJ
토큰: ##는        의존관계: VP_MOD
토큰: 뜨겁         의존관계: IP
토큰: ##게        의존관계: ROOT
토큰: 하나         의존관계: NP_SBJ
토큰: ##는        의존관계: VP_MOD
토큰: 차갑         의존관